# Deep Q-Network (DQN)
---
In this notebook, you will implement a DQN agent with OpenAI Gym's LunarLander-v2 environment.

### 1. Import the Necessary Packages

In [1]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

MODEL_NAME = "distributional"
FILE_NUM = 0

### 2. Instantiate the Environment and Agent

Initialize the environment in the code cell below.

In [ ]:
env = gym.make('LunarLander-v2')
env.seed(0)
print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space.n)

State shape:  (8,)
Number of actions:  4


Before running the next code cell, familiarize yourself with the code in **Step 2** and **Step 3** of this notebook, along with the code in `dqn_agent.py` and `model.py`.  Once you have an understanding of how the different files work together, 
- Define a neural network architecture in `model.py` that maps states to action values.  This file is mostly empty - it's up to you to define your own deep Q-network!
- Finish the `learn` method in the `Agent` class in `dqn_agent.py`.  The sampled batch of experience tuples is already provided for you; you need only use the local and target Q-networks to compute the loss, before taking a step towards minimizing the loss.

Once you have completed the code in `dqn_agent.py` and `model.py`, run the code cell below.  (_If you end up needing to make multiple changes and get unexpected behavior, please restart the kernel and run the cells from the beginning of the notebook!_)

You can find the solution files, along with saved model weights for a trained agent, in the `solution/` folder.  (_Note that there are many ways to solve this exercise, and the "solution" is just one way of approaching the problem, to yield a trained agent._)

In [ ]:
# random play score
scores = []
for n in range(100):
    score = 0
    state = env.reset()
    for j in range(1000):
        state, reward, done, _ = env.step(env.action_space.sample())
        score += reward
        if done:
            break
    scores.append(score)
print("Random Score: {} +- {} ({} trials)".format(np.mean(scores), np.std(scores), len(scores)))

Random Score: -174.93425466006954 +- 94.07815412561524 (100 trials)


In [ ]:
from dqn_agent import Agent

agent = Agent(state_size=8, action_size=4, seed=0)

# watch an untrained agent
state = env.reset()
score = 0
for j in range(200):
    action = agent.act(state)
    env.render()
    state, reward, done, _ = env.step(action)
    score += reward
    if done:
        break 
        
env.close()
print("Score:", score)

Score: -420.0730482249217


### 3. Train the Agent with DQN

Run the code cell below to train the agent from scratch.  You are welcome to amend the supplied values of the parameters in the function, to try to see if you can get better performance!

In [ ]:
def dqn(n_episodes=2000, max_t=1000,
        eps_start=1.0, eps_end=0.01, eps_decay=0.995,
        a_start=0., a_end=1.0,
        beta_start=0., beta_end=1.0,
        continue_after_solved=True,
        save_name="checkpoint_dueling_solved.pth"):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    
    prioritized = hasattr(agent, 'beta')  # if using prioritized experience replay, initialize beta
    if prioritized:
        print("Priority Used")
        agent.a = a_start
        agent.beta = beta_start
        a_increment = (a_end - a_start) / n_episodes
        beta_increment = (beta_end - beta_start) / n_episodes
    else:
        print("Priority Not Used")
        
    solved = False
    epi_str_max_len = len(str(n_episodes))
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            next_state, reward, done, _ = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        if prioritized:
            agent.a = min(a_end, agent.a + a_increment)
            agent.beta = min(beta_end, agent.beta + beta_increment)
        print('\rEpisode {:>{epi_max_len}d} | Current Score: {:>7.2f} | Average Score: {:>7.2f} | Epsilon: {:>6.4f}'\
              .format(i_episode, score, np.mean(scores_window), eps, epi_max_len=epi_str_max_len), end="")
        if prioritized:
            print(' | A: {:>6.4f} | Beta: {:>6.4f}'.format(agent.a, agent.beta), end='')
        print('          ', end='')
        if i_episode % 100 == 0:
            print('\rEpisode {:>{epi_max_len}} | Current Score: {:>7.2f} | Average Score: {:>7.2f} | Epsilon: {:>6.4f}'\
                  .format(i_episode, score, np.mean(scores_window), eps, epi_max_len=epi_str_max_len), end='')
            if prioritized:
                print(' | A: {:>6.4f} | Beta: {:>6.4f}'.format(agent.a, agent.beta), end='')
            print('          ')
        if not solved and np.mean(scores_window)>=200.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), save_name)
            solved = True
            if not continue_after_solved:
                break
    return scores

scores = dqn(n_episodes=3000, max_t=1000,
             eps_start=0., eps_end=0., eps_decay=0.,
             a_start=0.8, a_end=0.8,
             beta_start=0.0, beta_end=1.0,
             continue_after_solved=True,
             save_name="checkpoint_{}_solved{}.pth".format(MODEL_NAME, FILE_NUM))

# plot the scores
plt.rcParams['figure.facecolor'] = 'w'
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Priority Used
Episode  100 | Current Score:  -65.18 | Average Score: -109.64 | Epsilon: 0.0000 | A: 0.8000 | Beta: 0.0333          
Episode  200 | Current Score:  -46.85 | Average Score:   64.07 | Epsilon: 0.0000 | A: 0.8000 | Beta: 0.0667          
Episode  300 | Current Score:   -7.10 | Average Score:   78.65 | Epsilon: 0.0000 | A: 0.8000 | Beta: 0.1000          
Episode  400 | Current Score:  174.76 | Average Score:   50.73 | Epsilon: 0.0000 | A: 0.8000 | Beta: 0.1333          
Episode  500 | Current Score:  242.26 | Average Score:   95.49 | Epsilon: 0.0000 | A: 0.8000 | Beta: 0.1667          
Episode  600 | Current Score: -276.90 | Average Score:  101.03 | Epsilon: 0.0000 | A: 0.8000 | Beta: 0.2000          
Episode  700 | Current Score:  192.41 | Average Score:   73.92 | Epsilon: 0.0000 | A: 0.8000 | Beta: 0.2333          
Episode  800 | Current Score:   -4.59 | Average Score:   80.29 | Epsilon: 0.0000 | A: 0.8000 | Beta: 0.2667          
Episode  900 | Current Score:  231.15 | Av

In [ ]:
torch.save(agent.qnetwork_local.state_dict(), "checkpoint_{}_final{}.pth".format(MODEL_NAME, FILE_NUM))

In [6]:
agent.qnetwork_local.load_state_dict(torch.load("checkpoint_{}_final{}.pth".format(MODEL_NAME, FILE_NUM)))

### 4. Watch a Smart Agent!

In the next code cell, you will load the trained weights from file to watch a smart agent!

In [8]:
if hasattr(agent, 'noise'):
    agent.noise(False)

for i in range(10):
    state = env.reset()
    score = 0
    for j in range(1000):
        action = agent.act(state)
        env.render()
        state, reward, done, _ = env.step(action)
        score += reward
        if done:
            break
    print("Game {} Score: {} in {} steps".format(i, score, j + 1))

if hasattr(agent, 'noise'):
    agent.noise(True)
env.close()

Game 0 Score: 249.98635359973989 in 166 steps
Game 1 Score: 188.83143334284688 in 1000 steps
Game 2 Score: 273.1654934029272 in 221 steps
Game 3 Score: 244.99494942745116 in 171 steps
Game 4 Score: 272.1399594288923 in 200 steps
Game 5 Score: 192.10919093304733 in 350 steps
Game 6 Score: 270.2023976942239 in 169 steps
Game 7 Score: 293.39158700312123 in 210 steps
Game 8 Score: 259.39039660129225 in 167 steps
Game 9 Score: 242.49702623279921 in 191 steps


### 5. Explore

In this exercise, you have implemented a DQN agent and demonstrated how to use it to solve an OpenAI Gym environment.  To continue your learning, you are encouraged to complete any (or all!) of the following tasks:
- Amend the various hyperparameters and network architecture to see if you can get your agent to solve the environment faster.  Once you build intuition for the hyperparameters that work well with this environment, try solving a different OpenAI Gym task with discrete actions!
- You may like to implement some improvements such as prioritized experience replay, Double DQN, or Dueling DQN! 
- Write a blog post explaining the intuition behind the DQN algorithm and demonstrating how to use it to solve an RL environment of your choosing.  